In [1]:
import json
from pyspark.mllib.util import MLUtils
from collections import Counter
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.classification import LabeledPoint

In [2]:
def add_bi_tri_gramms(doc):
    words = doc['Features']
    bigramms = ['{} {}'.format(w_0, w_1) for w_0, w_1 in zip(words[:-1], words[1:])]
    trigramms = ['{} {} {}'.format(w_0, w_1, w_2) for w_0, w_1, w_2 in zip(words[:-2], words[1:-1], words[2:])]
    doc['Features'].extend(bigramms)
    doc['Features'].extend(trigramms)
    return doc    

In [3]:
temp_data = sc.textFile('hdfs://master:54310/exp_f/clean') \
    .map(json.loads).map(add_bi_tri_gramms)

In [4]:
word_WL = temp_data.flatMap(lambda x: set(x['Features'])) \
    .map(lambda w: (w, 1)) \
    .reduceByKey(int.__add__) \
    .filter(lambda wc: wc[1] > 24) \
    .map(lambda wc: wc[0])  \
    .collect()

In [5]:
len(word_WL)

226119

In [6]:
word_WL_br = sc.broadcast(set(word_WL))

In [7]:
def filter_words(doc):
    doc['Features'] = [w for w in doc['Features'] if w in word_WL_br.value]
    return doc

In [8]:
clean_rdd = temp_data.map(filter_words)

In [11]:
clean_rdd.map(json.dumps).repartition(6).saveAsTextFile('hdfs://master:54310/exp_f/2_3_gramms')